## TRABALHO DE IAA002 - Linguagem de Programação Aplicada
---

### Importação das Bibliotecas

In [2]:
# Biblioteca Pandas - Manipulação de dados
import pandas as pd
# Biblioteca Seaborn - Criação de gráficos
import seaborn as sns
# Biblioteca Matplotlib - Criação de gráficos
import matplotlib.pyplot as plt

# OPCIONAL - Biblioteca para ignorar mensagens de warning (aviso) ao rodar uma célula de código
import warnings
warnings.filterwarnings('ignore')

# Bibliotecas de machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder

# Métricas de avaliação dos modelos
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

---
### 1 Análise exploratória dos dados

#### a. Carregue a base de dados media_precos_carros_brasil.csv

In [19]:
# Função read_csv para importar os dados da pasta do computador
dados = pd.read_csv('database/precos_carros_brasil.csv')

In [20]:
# nº de linhas e colunas da tabela
linhas_inicial = dados.shape
linhas_inicial

(267542, 11)

#### b.1. Verifique se há valores faltantes nos dados. 

In [21]:

# Verificando se existem valores faltantes nos dados 
dados.isna().sum()

year_of_reference     65245
month_of_reference    65245
fipe_code             65245
authentication        65245
brand                 65245
model                 65245
fuel                  65245
gear                  65245
engine_size           65245
year_model            65245
avg_price_brl         65245
dtype: int64

In [6]:
# Como em todas as colunas há o exato mesmo número de itens faltantes
# verificamos aqui se não são na verdade linhas com todos os dados faltentes
dados.isna().head()

,year_of_reference,month_of_reference,fipe_code,authentication,brand,model,fuel,gear,engine_size,year_model,avg_price_brl
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False


##### b.2. Caso haja, escolha uma tratativa para resolver o problema de valores faltantes.

In [29]:
# A melhor trarativa no caso é excluir as linhas com valores nulos
# O método dropna() remove as linhas que contêm valores nulos.
# axis=0 indica que estamos removendo as linhas com valores nulos.
dados.dropna(axis=0, inplace=True)


In [31]:
# Aqui verificamos se foram exclidos exatamente o valor acima das linhas com itens nulos.
# se o valor for igual, quer dizer só existiam linhas inteiras sem dados.

linhas_final = dados.shape
verifica_linhas = linhas_inicial[0] - linhas_final[0]
verifica_linhas


65245

In [32]:
#Verificando novamente se existem colunas com valores nulos.
dados.isna().any()

year_of_reference     False
month_of_reference    False
fipe_code             False
authentication        False
brand                 False
model                 False
fuel                  False
gear                  False
engine_size           False
year_model            False
avg_price_brl         False
dtype: bool

#### c. Verifique se há valores duplicados nos dados


In [33]:
# Verificando se temos valores duplicados
dados.duplicated().sum() 

2

# dúvida, será que precisa remover o valor duplicado?

In [34]:
# Removendo valores duplicados
dados.drop_duplicates(inplace=True)

#### d.1. Crie duas categorias, para separar colunas numéricas e categóricas

In [35]:
# Criando categorias para separar colunas numéricas e categóricas: facilita a AED
numericas_cols = [col for col in dados.columns if dados[col].dtype != 'object']
categoricas_cols = [col for col in dados.columns if dados[col].dtype == 'object']

#### d.2. Imprima o resumo de informações das variáveis numéricas e categóricas (estatística descritiva dos dados)

In [36]:
# Resumo das variáveis numéricas - Imprime alguns valores de medidas de tendências centrais 
dados[numericas_cols].describe()

,year_of_reference,year_model,avg_price_brl
count,202295.000000,202295.000000,202295.000000
mean,2021.564695,2011.271514,52756.765713
std,0.571904,6.376241,51628.912116
min,2021.000000,2000.000000,6647.000000
25%,2021.000000,2006.000000,22855.000000
50%,2022.000000,2012.000000,38027.000000
75%,2022.000000,2016.000000,64064.000000
max,2023.000000,2023.000000,979358.000000


In [37]:
# Resumo das variáveis categóricas - Imprime alguns valores de estatística descritiva
dados[categoricas_cols].describe()

,month_of_reference,fipe_code,authentication,brand,model,fuel,gear,engine_size
count,202295,202295,202295,202295,202295,202295,202295,202295
unique,12,2091,202295,6,2112,3,2,29
top,January,003281-6,cfzlctzfwrcp,Fiat,Palio Week. Adv/Adv TRYON 1.8 mpi Flex,Gasoline,manual,"1,6"
freq,24260,425,1,44962,425,168684,161883,47420


#### e. Imprima a contagem de valores por modelo (model) e marca do carro (brand)


In [38]:
# Contagem de valores por categoria de 'Modelo'
dados['model'].value_counts()

model
Palio Week. Adv/Adv TRYON 1.8 mpi Flex    425
Focus 1.6 S/SE/SE Plus Flex 8V/16V 5p     425
Focus 2.0 16V/SE/SE Plus Flex 5p Aut.     400
Saveiro 1.6 Mi/ 1.6 Mi Total Flex 8V      400
Corvette 5.7/ 6.0, 6.2 Targa/Stingray     375
                                         ... 
STEPWAY Zen Flex 1.0 12V Mec.               2
Saveiro Robust 1.6 Total Flex 16V CD        2
Saveiro Robust 1.6 Total Flex 16V           2
Gol Last Edition 1.0 Flex 12V 5p            2
Polo Track 1.0 Flex 12V 5p                  2
Name: count, Length: 2112, dtype: int64

In [39]:
# Contagem de valores por categoria de 'Marca'
dados['brand'].value_counts()

brand
Fiat               44962
VW - VolksWagen    44312
GM - Chevrolet     38590
Ford               33150
Renault            29191
Nissan             12090
Name: count, dtype: int64

#### f.Dê um breve explicação (máximo de quatro linhas) sobre os principais resultados encontrados na Análise Exploratória dos dados
# Parei aqui kkkkkk